In [1]:
import csv
import math
import pandas as pd
from datetime import datetime

In [2]:
# for keeping track of runtime
# for reference, it ran 2min on the 7 month 31.5GB 2017 dataset
datetime.now()

datetime.datetime(2019, 3, 27, 20, 44, 29, 141564)

# Import here

In [3]:
day_df = pd.read_csv("14_Taxi_Trips.csv", sep = ",")

In [4]:
# lean_df only keeps the 2 columns with information we use
lean_df = day_df[["Trip Start Timestamp", "Pickup Community Area"]]
lean_df = lean_df.dropna(subset = ["Pickup Community Area"])
lean_df["Pickup Community Area"] = lean_df["Pickup Community Area"].astype('int')

In [5]:
# parse datetime and round to hourly level
date_df = lean_df
date_df["Trip Start Timestamp"] = pd.to_datetime(lean_df["Trip Start Timestamp"], \
                                                 format='%m/%d/%Y %I:%M:%S %p').dt.floor('H');
date_df["dummy"] = 0 # for groupby operations later

In [6]:
# create a full cartesian product of all community areas and all hourly timestamp
# alltimes_df used to fill in 0's as Count for community areas that did not
# appear for certain hours
alltimes_df = pd.DataFrame(date_df.drop_duplicates(["Trip Start Timestamp"])["Trip Start Timestamp"])
alltimes_df["Key"] = 0
allcomms_df = pd.DataFrame({"Community Area": [i for i in range(1,78)]})
allcomms_df["Key"] = 0
fillin_df = alltimes_df.merge(right = allcomms_df, on = "Key").drop(["Key"], axis = 1)

In [7]:
# count pickup number for each community each hour
aggregate_df = date_df.groupby(["Trip Start Timestamp","Pickup Community Area"])\
                ["dummy"].count().reset_index()\
                .rename(columns={"dummy":"Count"})

In [8]:
# use fillin_df to fill in 0's for non-appearing community areas
fullentry_df = fillin_df.merge(right = aggregate_df, \
                               left_on = ["Trip Start Timestamp", "Community Area"], \
                               right_on = ["Trip Start Timestamp", "Pickup Community Area"],\
                               how="left").drop(["Pickup Community Area"], axis = 1)
fullentry_df["Count"] = fullentry_df["Count"].fillna(0).astype('int')

In [9]:
# add in (transformed) fields about time
date_df = fullentry_df
date_df["dayofyear"] = date_df["Trip Start Timestamp"].dt.dayofyear;
date_df["dayofyear_sin"] = date_df["dayofyear"].map(lambda x: round(math.sin(x/365*2*math.pi),3));
date_df["dayofyear_cos"] = date_df["dayofyear"].map(lambda x: round(math.cos(x/365*2*math.pi),3));

date_df["hour"] = date_df["Trip Start Timestamp"].dt.hour;
date_df["hour_sin"] = date_df["hour"].map(lambda x: round(math.sin(x/24*2*math.pi),2));
date_df["hour_cos"] = date_df["hour"].map(lambda x: round(math.cos(x/24*2*math.pi),2));

date_df["weekday"] = date_df["Trip Start Timestamp"].dt.dayofweek #Monday = 0, Sunday = 6

In [10]:
# add in coordinates for each community area
mapping_df = pd.read_csv("community_mapping.csv")
mapped_df = date_df.merge(right = mapping_df, left_on = "Community Area",\
                          right_on = "Area Number", how = "left")\
                         .drop(["Area Number"], axis = 1)

In [11]:
# delete extra bits to save memory
clean_df = mapped_df
clean_df["Trip Start Timestamp"] = mapped_df["Trip Start Timestamp"].dt.strftime('%Y/%m/%d %H')
clean_df = clean_df.rename(columns = {"Community Area": "Pickup Community Area"})

# Export here

In [12]:
clean_df.to_csv("14_Taxi_Trips.csv", index = False)

In [13]:
# for keeping track of runtime
datetime.now()

datetime.datetime(2019, 3, 27, 20, 52, 57, 342975)